In [0]:
import requests # Peticiones HTTP

## Requests intro
---

In [0]:
url = 'https://www.pagina12.com.ar/'
p12 = requests.get(url) # Petición por método GET

In [37]:
p12.status_code # 200 = Bien hecho

200

In [0]:
p12_html = p12.text # Texto HTML (string)

In [0]:
p12_content = p12.content # Texto HTML (bytes)

In [40]:
p12.headers # HTTP response headers

{'Date': 'Thu, 12 Mar 2020 04:00:59 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=d56157cb05bcbc1bee0eb0c65b0619d0c1583985659; expires=Sat, 11-Apr-20 04:00:59 GMT; path=/; domain=.pagina12.com.ar; HttpOnly; SameSite=Lax', 'Vary': 'Accept-Encoding', 'X-DNS-Prefetch-Control': 'off', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'X-Download-Options': 'noopen', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'X-Backend': 'prod_frontend_1', 'X-Backend-TTL': '180.000', 'X-Type': 'Dynamic URI', 'Age': '0', 'grace': '86400.000 none', 'ttl': '119.874', 'x-debug': '', 'X-Instance': 'cache-front-prod-varnish-76c5f88cf6-6j89c', 'x-restarts': '0', 'X-Cache': 'HIT (1)', 'CF-Cache-Status': 'DYNAMIC', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Server': 'cloudflare', 'CF-RAY': '572a9c80ca9fe392-ATL', 'Conten

In [41]:
p12.request.headers # HTTP request headers

{'User-Agent': 'python-requests/2.21.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [42]:
p12.request.method # request method

'GET'

In [43]:
p12.request.url # request url

'https://www.pagina12.com.ar/'

## Intro BeautifulSoup
---

In [0]:
from bs4 import BeautifulSoup # HTML parser

In [0]:
s = BeautifulSoup(p12.text, 'lxml') # Parser

In [46]:
type(s)

bs4.BeautifulSoup

In [47]:
print(s.prettify()[:80]) # HTML estructurado

<!DOCTYPE html>
<html class="no-js">
 <head>
  <meta charset="utf-8"/>
  <title>


In [0]:
ul_hot_sections = s.find('ul', attrs={'class': 'hot-sections'}) # Devuelve el primer elemento que haga match

In [0]:
sections = ul_hot_sections.find_all('li') # Devuelve todos los elementos que hagan match (list)

In [50]:
for section in sections:
  print(section.get_text()) # Devuelve todos los textos hijos

El país
Economía
Sociedad
Cultura y Espectáculos
El mundo
Deportes
Psicología
Contratapa


## Get info
---

In [51]:
sections[0].a.get('href') # Devuelve el valor de un atributo

'https://www.pagina12.com.ar/secciones/el-pais'

In [52]:
link_secciones = [section.a.get('href') for section in sections]
link_secciones

['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/el-mundo',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/psicologia',
 'https://www.pagina12.com.ar/secciones/contratapa']

In [0]:
s = requests.get(link_secciones[0])
s_section = BeautifulSoup(s.text, 'lxml')

In [0]:
featured_article = s_section.find('div', attrs={ 'class':'featured-article__container' })

In [55]:
featured_article.a.get('href')

'https://www.pagina12.com.ar/252265-desafios-de-la-educacion-superior'

In [0]:
article_list = s_section.find('ul', attrs={ 'class':'article-list' })

In [57]:
# Reto
def get_articles_links_from_section(sectionBS):
  links = []

  # Featured article
  links.append(sectionBS.find('div', attrs={ 'class':'featured-article__container'}).a.get('href'))

  # "Normal" articles
  for article in sectionBS.find('ul', attrs={ 'class':'article-list' }).find_all('li'):
    if article.get_text():
      links.append(article.find('h2').a.get('href'))

  return list(set(links))

lista_notas = get_articles_links_from_section(s_section)
lista_notas

['https://www.pagina12.com.ar/252030-coronavirus-licencia-de-14-dias-para-quienes-lleguen-del-ext',
 'https://www.pagina12.com.ar/252406-le-impusieron-una-multa-a-larreta-por-no-cumplir-con-un-plan',
 'https://www.pagina12.com.ar/252251-alberto-fernandez-volvio-a-dar-clases-en-la-facultad-de-dere',
 'https://www.pagina12.com.ar/252422-legisladores-y-empleados-del-congreso-en-cuarentena',
 'https://www.pagina12.com.ar/252399-el-senado-sesionara-para-convertir-en-ley-las-limitaciones-a',
 'https://www.pagina12.com.ar/252222-ramos-padilla-denuncia-que-en-comodoro-py-buscan-beneficiar-',
 'https://www.pagina12.com.ar/252408-dibujos-urgentes-retratos-de-genocidas-y-testigos-de-juicios',
 'https://www.pagina12.com.ar/252226-alberto-fernandez-dijo-que-le-preocupan-situaciones-de-la-ju',
 'https://www.pagina12.com.ar/252310-la-justicia-investiga-un-multimillonaro-desvio-de-acero-de-l',
 'https://www.pagina12.com.ar/252379-lagrimas-y-alegrias',
 'https://www.pagina12.com.ar/252281-el-aborto-div

## Error Handling
---

In [58]:
# status_code == 200

r = requests.get(url)

if r.status_code == 200:
  # Use the response
  print('Yeih!!')
else:
  # Show the error
  print('Oh no!')

Yeih!!


In [59]:
# Revisar si se obtuvo respuesta del servidor

url_mala = url.replace('2', '3') # https://www.pagina13.com.ar/

try:
  response = requests.get(url_mala)
  print(response)
except Exception as e:
  print('Error')
  print(e)

Error
HTTPSConnectionPool(host='www.pagina13.com.ar', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f2e6e561e10>: Failed to establish a new connection: [Errno -2] Name or service not known',))


In [60]:
# Revisar si un tag existió

try:
  text = featured_article.somehtmltag.get_text()
  print(text)
except Exception as e:
  print('Error')
  print(e)

Error
'NoneType' object has no attribute 'get_text'


## Get info
---

In [61]:
url_nota = lista_notas[0]
print(url_nota)

https://www.pagina12.com.ar/252030-coronavirus-licencia-de-14-dias-para-quienes-lleguen-del-ext


In [62]:
try:
  nota = requests.get(url_nota)
  if nota.status_code == 200:
    s_nota = BeautifulSoup(nota.text, 'lxml')

    # Title
    title = s_nota.find('div', attrs={ 'class':'article-title' }).get_text()
    date = s_nota.find('span', attrs={ 'pubdate':'pubdate' }).get('datetime')

    print(title)
    print(date)
except Exception as e:
  print('Error')
  print(e)
  print('\n')

El estornudo Rodríguez Larreta y el coronavirus
2020-03-11


## Media content
---

In [63]:
media = s_nota.find('div', attrs={ 'class':'article-main-media-image' })
images = media.find_all('img')

if images:
  image = images[-1]
  image_src = image.get('data-src')
  print(image_src)
else:
  print('No images found')

https://images.pagina12.com.ar/styles/focal_3_2_960x640/public/media/articles/56147/coronavirus_0.jpg?itok=X-jml-oQ


In [0]:
from IPython.display import Image

In [0]:
img_req = requests.get(image_src)

if img_req.status_code == 200:
  Image(img_req.content)
else:
  print('Error')

## Final scraper
---

In [0]:
def obtener_info(s_nota):
    
    # Creamos un diccionario vacío para poblarlo con la información
    ret_dict = {}
    
    # Extraemos la fecha
    fecha = s_nota.find('span', attrs={'pubdate':'pubdate'})
    if fecha:
        ret_dict['fecha'] = fecha.get('datetime')
    else:
        ret_dict['fecha'] = None
    
    # Extraemos el título
    titulo = s_nota.find('div', attrs={'class':'article-title'})
    if titulo:
        ret_dict['titulo'] = titulo.text
    else:
        ret_dict['titulo'] = None

    # Extraemos la volanta
    volanta = s_nota.find('div', attrs={'class':'article-prefix'})
    if volanta:
        ret_dict['volanta'] = volanta.get_text()
    else:
        ret_dict['volanta'] = None
    
    # Extraemos el copete
    copete = s_nota.find('div', attrs={'class':'article-summary'})
    if copete:
        ret_dict['copete'] = copete.get_text()
    else:
        ret_dict['copete'] = None
    
    autor = s_nota.find('div', attrs={'class':'article-author'})
    if autor:
        ret_dict['autor'] = autor.a.get_text()
    else:
        ret_dict['autor'] = None
    
    # Extraemos la imagen
    media = s_nota.find('div', attrs={'class':'article-main-media-image'})
    if media:
        imagenes = media.find_all('img')
        if len(imagenes) == 0:
            print('no se encontraron imágenes')
        else:
            imagen = imagenes[-1]
            img_src = imagen.get('data-src')
            try:
                img_req = requests.get(img_src)
                if img_req.status_code == 200:
                    ret_dict['imagen'] = img_req.content
                else:
                    ret_dict['imagen'] = None
            except:
                print('No se pudo obtener la imagen')
    else:
        print('No se encontró media')
    # Extraemos el cuerpo de la nota
    cuerpo = s_nota.find('div', attrs={'class':'article-text'})
    if cuerpo:
        ret_dict['texto'] = cuerpo.get_text()
    else:
        ret_dict['texto'] = None
    
    return ret_dict

In [67]:
obtener_info(s_nota)

{'autor': 'Ezequiel Boetti',
 'copete': None,
 'fecha': '2020-03-11',
 'imagen': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc2\x00\x11\x08\x02\x80\x03\xc0\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1b\x00\x00\x02\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x00\x03\x04\x05\x06\x07\xff\xc4\x00\x19\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\xff\xda\x00\x0c\x03\x01\x00\x02\x10\x03\x10\x00\x00\x01\xeb\xd9\xa6r\xe8\xaf$I$\x00@\x01\x02WmeXw\xe3\xaej\xdd\x8f\xa699\xe9\x06\x89C\x16\x9e\x8fw\x17\xc8Mx\xf4h\xb0 \xad\x8cT\xca\xd1H`\x83E!\x82\x1a}G\x8d9z\x0e\x12\x1av\xe8u\xe5\xf3\x02\xda(\xa9\x96\t\x00`\x81

In [0]:
def scrape_nota(url):
  try:
    nota = requests.get(url)
  except Exception as e:
    print(f'Error scrapeando {url}')
    print(e)
    print('\n')
    return None

  if nota.status_code != 200:
    print(f'Error scrapeando {url}')
    print(f'Status code = {nota.status_code}')
    print('\n')
    return None
  
  s_nota = BeautifulSoup(nota.text, 'lxml')

  ret_dict = obtener_info(s_nota)
  ret_dict['url'] = url

  return ret_dict

In [69]:
scrape_nota(url_nota)

{'autor': 'Ezequiel Boetti',
 'copete': None,
 'fecha': '2020-03-11',
 'imagen': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc2\x00\x11\x08\x02\x80\x03\xc0\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1b\x00\x00\x02\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x00\x03\x04\x05\x06\x07\xff\xc4\x00\x19\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\xff\xda\x00\x0c\x03\x01\x00\x02\x10\x03\x10\x00\x00\x01\xeb\xd9\xa6r\xe8\xaf$I$\x00@\x01\x02WmeXw\xe3\xaej\xdd\x8f\xa699\xe9\x06\x89C\x16\x9e\x8fw\x17\xc8Mx\xf4h\xb0 \xad\x8cT\xca\xd1H`\x83E!\x82\x1a}G\x8d9z\x0e\x12\x1av\xe8u\xe5\xf3\x02\xda(\xa9\x96\t\x00`\x81